In [1]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_excel('/home/yaroslav/progs/digitdep_hack/data/info/buyer.xlsx')
df.columns = ['code', 'addr']

In [136]:
cities = pd.read_csv('geodata/table_geo.csv')
cities.city = cities.city.apply(lambda x: x.lower())
cities

,city,region,geo
0,абаза,Хакасия,"52.651662,90.088581"
1,абакан,Хакасия,"53.721152,91.442396"
2,абдулино,Оренбургская область,"53.683962,53.654989"
3,абинск,Краснодарский край,"44.866237,38.151288"
4,агидель,Башкортостан,"55.899043,53.933439"
...,...,...,...
1120,ярославль,Ярославская область,"57.479957,39.752643"
1121,ярцево,Смоленская область,"55.066123,32.688292"
1122,ясногорск,Тульская область,"54.479293,37.693354"
1123,ясный,Оренбургская область,"51.035711,59.874088"


In [55]:
import re

IGNORED_WORDS = {"Россия", "РФ", "ХМАО", "ЯНАО", "МО"}

def extract_city(address):
    # Регулярное выражение для поиска населенного пункта
    city_pattern = (
        r'\b(?:г\.|город\s+)?'  # Опционально "г." или "город"
        r'([А-ЯЁ][а-яё\-]+(?:\s+[А-ЯЁ][а-яё\-]+)*)'  # Название города (включая составные)
        r'\b'  # Границы слова
    )
    
    matches = re.findall(city_pattern, address)
    # Фильтрация игнорируемых слов
    filtered_matches = [m for m in matches if m not in IGNORED_WORDS]

    return filtered_matches[0] if filtered_matches else None


# df.addr.apply(lambdaextract_city(addr) for addr in addresses if extract_city(addr)]

# # Вывод всех уникальных городов
# unique_cities = set(cities)
# print("Найденные города:", unique_cities)


In [56]:
idx = 0

In [97]:
import string

def custom_strip(text):
    # Определяем набор символов для удаления
    strip_chars = string.whitespace + string.punctuation
    # Удаляем указанные символы с начала и конца строки
    return text.strip(strip_chars)

def clear_addr(addr):
    addr = addr.lower() if isinstance(addr, str) else addr
    list_r = list(map(custom_strip, addr.replace('город', 'г.').replace(',', ' ').replace('.', ' ').split()))
    if 'г' in list_r:
        res_idx = list_r.index('г')
    else:
        res_idx = 1
    return res_idx, list_r

проблема: 20 городов с неуникальным названием

In [139]:
cities_set = set(cities.city)

In [184]:
cities.query('city == "абаза"').geo.values[0]

'52.651662,90.088581'

In [380]:
from natasha import (
    MorphVocab,
    AddrExtractor,
)

def apply_finder(x):
    c_id, list_r = clear_addr(x.lower())
    if c_id + 1 < len(list_r):
        town = list_r[c_id + 1]
    else:
        town = 'nan'
    # idx += 1
    if town in cities_set:
        return cities.query(f'city == "{town}"').geo.values[0]
    else:
        return 'nan'


def finder_natasha(x):
    global c
    global green
    matches = addr_extractor(x)
    town = 'nan'
    c += 1
    for match in matches:
        if match.fact.type == 'город':
            town = match.fact.value
            break
    if town.lower() in cities_set:
        green += 1
        print(c, green, end='\r')
        res = cities.query(f'city == "{town.lower()}"').geo.values
        if res.shape[0] > 1:
            print(f'{x=}, {res=}')
            return 'nan'
        return res[0]

In [189]:
df['geo'] = 'nan'

In [201]:
df['geo'] = df.addr.apply(apply_finder)

In [381]:
morph_vocab = MorphVocab()
addr_extractor = AddrExtractor(morph_vocab)
c = 0
green = 0
print(df.shape[0])
df['geo_v2'] = df.addr.apply(finder_natasha)

1856
x='652780, Кемеровская обл., г. Гурьевск, пер. Нефтяников, д. 2 ', res=array(['54.770401,20.608359', '54.284748,85.932059'], dtype=object)
x='ХМАО, г. Радужный, Тагринское м/р, 133 куст ', res=array(['55.995674,40.33194', '62.135232,77.464'], dtype=object)
x='652785, Кемеровская обл.-Кузбасс, г. Гурьевск, пер. Нефтяников, д. 2 ', res=array(['54.770401,20.608359', '54.284748,85.932059'], dtype=object)
x='453430, Респ. Башкортостан, г. Благовещенск, ул. Чапаева, д. 75', res=array(['55.049854,55.955224', '50.257456,127.534611'], dtype=object)


In [391]:
doubles = ['652780, Кемеровская обл., г. Гурьевск, пер. Нефтяников, д. 2 ', 'ХМАО, г. Радужный, Тагринское м/р, 133 куст ', 
           '652785, Кемеровская обл.-Кузбасс, г. Гурьевск, пер. Нефтяников, д. 2 ', '453430, Респ. Башкортостан, г. Благовещенск, ул. Чапаева, д. 75']

In [405]:
cities[cities.city == 'благовещенск']

,city,region,geo
108,благовещенск,Башкортостан,"55.049854,55.955224"
109,благовещенск,Амурская область,"50.257456,127.534611"


In [404]:

for region in cities[cities.city == 'благовещенск'].region:
    matches = addr_extractor(f'{region}')
    for match in matches:
        print(f'{match.fact.type}: {match.fact.value}')

область: Амурская


In [428]:
# df[df.addr == '453430, Респ. Башкортостан, г. Благовещенск, ул. Чапаева, д. 75'].geo

1506    nan
Name: geo, dtype: object

In [427]:
# df.loc[1506, 'geo'] = 'nan'

In [402]:
matches = addr_extractor(doubles[3])
for match in matches:
    print(f'{match.fact.type}: {match.fact.value}')
    if match.fact.type == 'город':
        town = match.fact.value
        break

индекс: 453430
республика: Башкортостан
город: Благовещенск


In [409]:
df[['addr', 'geo_v2']].head(30)

,addr,geo_v2
0,"644099, Россия, г. Омск. ул. Фрунзе, д.54","54.989347,73.368221"
1,"РФ, г. Омск, пр. Губкина, д.1","54.989347,73.368221"
2,"г. Екатеринбург, ул. Сакко и Ванцетти, д.61","56.838011,60.597474"
3,"Россия, 190900, Санкт-Петербург, box 1283",None
4,"г. Москва, Капотня, 2 квартал, д.1, корп. 3","55.755819,37.617644"
5,"628690, Россия, ХМАО-Югра, г.Мегион, пгт.Высокий","61.03289,76.102621"
6,"г. Омск, Доковская, д. 23","54.989347,73.368221"
7,"РФ, 629800, Ямало-Нененский автономный округ, ...","63.201805,75.450938"
8,"Россия, 191014, г. Санкт-Петербург, Виленский ...","59.938676,30.314494"
9,"628011, ХМАО-Югра, г. Ханты-Мансийск, ул.Ленин...","61.026144,69.095798"


In [385]:
(df.geo_v2 =='nan').sum() + df.geo_v2.isna().sum(), df.shape[0]

(683, 1856)

In [202]:
(df.geo == 'nan').sum(), df.shape[0]

(729, 1856)

In [222]:
nan_iter = iter(df[df.geo == 'nan'].values)

In [205]:
post_codes = pd.read_csv('geodata/post_codes.csv')
post_codes.columns = list(map(str.lower, post_codes.columns.tolist()))
post_codes = post_codes[['index', 'region', 'autonom', 'area', 'city', 'city_1']]
for col in ['region', 'autonom', 'area', 'city', 'city_1']:
    post_codes[col] = post_codes[col].apply(lambda x: x.lower() if isinstance(x, str) else None)

In [252]:
next(nan_iter)

array([20000537, 'ст. Лабытнанги Северной ж.д. код станции:  289904',
       'nan'], dtype=object)

In [255]:
df.shape[0], df[df.geo == 'nan'].values.shape[0]

(1856, 729)

In [258]:
df.geo.value_counts()[:10]

geo
nan                    729
59.938676,30.314494    153
55.755819,37.617644    116
54.989347,73.368221     78
63.201805,75.450938     62
57.152985,65.541227     49
51.768205,55.097        40
68.970663,33.074918     37
56.838011,60.597474     36
61.03289,76.102621      31
Name: count, dtype: int64

In [240]:
post_codes[post_codes['index'] == 190000]

,index,region,autonom,area,city,city_1
9145,190000,санкт-петербург,None,None,None,None


In [269]:
from src.utils.main_parser import GeoProcessor

In [270]:
geopc = GeoProcessor()

In [276]:
df.loc[16, 'geo'] = 'nan'

In [278]:
from tqdm import tqdm

In [431]:
df = df.drop(columns=['geo_v2'])

In [432]:
save_path = '/home/yaroslav/progs/digitdep_hack/data/info/buyer.csv'
browser = geopc.init_browser_ymaps()

count_idx = 0
for (df_idx, (code, addr, geo)) in tqdm(zip(df[df.geo == 'nan'].index.tolist(), df[df.geo == 'nan'].values), total=df[df.geo == 'nan'].values.shape[0]):
    try:
        if geo == 'nan':
            count_idx += 1
            parse_result = geopc.process_v2(addr, browser=browser)
            if isinstance(parse_result, tuple):
                parse_result = ','.join(map(str, parse_result))
            df.loc[df_idx, 'geo'] = parse_result
            if count_idx % 5 == 0:
                df.to_csv(save_path, index=False, encoding='UTF-8')
    except Exception as e:
        print(str(e), f' {parse_result=}, {type(parse_result)}')

100%|██████████| 219/219 [19:39<00:00,  5.39s/it]


In [433]:
count_idx = 0
for (df_idx, (code, addr, geo)) in tqdm(zip(df[df.geo == 'init'].index.tolist(), df[df.geo == 'init'].values), total=df[df.geo == 'init'].values.shape[0]):
    try:
        if geo == 'init':
            count_idx += 1
            parse_result = geopc.process_v2(addr, browser=browser)
            if isinstance(parse_result, tuple):
                parse_result = ','.join(map(str, parse_result))
            df.loc[df_idx, 'geo'] = parse_result
            if count_idx % 5 == 0:
                df.to_csv(save_path, index=False, encoding='UTF-8')
    except Exception as e:
        print(str(e), f' {parse_result=}, {type(parse_result)}')

100%|██████████| 45/45 [04:04<00:00,  5.42s/it]


In [440]:
values_iter = iter(df[df.geo == 'nan'].addr.values)

In [497]:
next(values_iter)

'ст. Коротчаево Свердловской ж/д, код 799101 '

In [438]:
df.to_csv(save_path, index=False, encoding='UTF-8')